# My Personal Notes For Learning ChatGPT API
### https://github.com/mmrzax

In [1]:
import os
import openai
import requests
import json

In [2]:
# get openai api key from enviroment
openai.api_key = os.getenv("OPENAI_API_KEY")
# print(openai.api_key)

In [3]:
def get_current_weather(location: str, unit: str = "celcius"):
    units: dict[str, str] = {"celsius": "m", "fahrenheit": "u"}
    # r = requests.get(f"https://wttr.in/{location}?{units.get(unit)}&format=3")
    r = requests.get(f"https://wttr.in/{location}?{units.get(unit)}&format=4")
    return r.text
print(get_current_weather("BandarAbbas"))

BandarAbbas: ⛅️  🌡️+30°C 🌬️→7km/h



In [4]:
def chatgpt():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "آب و هوای شهر بندرعباس هم اکنون چگونه است؟"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "آب و هوا کنونی را برای مکان داده شده دریافت کن",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "نام مکان به زبان انگلیسی، برای مثال Tehran",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        
        res = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
            stream=True,
        )  # get a new response from GPT where it can see the function response
        return res

r = chatgpt()
chunks_of_response = []
msgs = []
for chunk in r:
    chunks_of_response.append(chunk)  # save the event response
    chunk_message = chunk['choices'][0]['delta']  # extract the message
    msgs.append(chunk_message)  # save the message
    print(chunk_message.get("content", ""), end="")

در حال حاضر آب و هوای شهر بندرعباس به صورت ابری است و دمای هوا حدود +30 درجه سانتیگراد است. سرعت باد نیز حدود 7 کیلومتر در ساعت است.

In [ ]:
full_reply_content = ''.join([m.get('content', '') for m in msgs])
print(full_reply_content)

In [ ]:
rr = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "فصل آخر attack on titan چه شد؟"}
    ],
    stream=True,
)
for chunk in rr:
    chunks_of_response.append(chunk)  # save the event response
    chunk_message = chunk['choices'][0]['delta']  # extract the message
    msgs.append(chunk_message)  # save the message
    print(chunk_message.get("content", ""), end="")